In [10]:
# Spark init#save the files in the g drive folder - first two commands
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!wget --continue https://drive.google.com/file/d/1b-Z0Tt--ioLGhPTNwHVKyYY7mS7ynRJs/view?usp=sharing   -O /content/drive/MyDrive/data/zip_code_market_tracker.tsv000
housing_df_raw = spark.read.csv('/content/drive/MyDrive/data/zip_code_market_tracker.tsv000', sep='\t',header=True)

--2022-04-25 02:57:14--  https://drive.google.com/file/d/1b-Z0Tt--ioLGhPTNwHVKyYY7mS7ynRJs/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.250.145.100, 142.250.145.101, 142.250.145.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.145.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/drive/MyDrive/data/zip_code_market_tracker.tsv000’

/content/drive/MyDr     [  <=>               ]  64.86K   303KB/s    in 0.2s    

2022-04-25 02:57:15 (303 KB/s) - ‘/content/drive/MyDrive/data/zip_code_market_tracker.tsv000’ saved [3028826328]



In [13]:
housing_trimmed_df = housing_df_raw.select('period_begin','period_end','property_type','property_type_id','state','state_code','region','median_sale_price','median_ppsf')

In [14]:
from pyspark.sql.functions import year,quarter,when,split,col

housing_transformed = housing_trimmed_df.withColumn('period_begin_year',year('period_begin')).withColumn('period_begin_quarter',quarter('period_begin'))
housing_transformed_2 = housing_transformed.withColumn('median_sales_2012', when(housing_transformed.period_begin_year== '2012',housing_transformed.median_sale_price).cast('float')).withColumn('median_ppsf_2012', when(housing_transformed.period_begin_year== '2012',housing_transformed.median_ppsf).cast('float')).withColumn('median_sales_2020', when(housing_transformed.period_begin_year== '2020',housing_transformed.median_sale_price).cast('float')).withColumn('median_ppsf_2020', when(housing_transformed.period_begin_year== '2020',housing_transformed.median_ppsf).cast('float'))
housing_transformed_3 = housing_transformed_2.select(housing_transformed_2.region.alias('zip'),housing_transformed_2.property_type_id.alias('type_id'),'median_sales_2012','median_ppsf_2012','median_sales_2020','median_ppsf_2020').fillna(0)
housing_transformed_4 = housing_transformed_3.groupBy('zip','type_id').sum()
housing_unique = housing_trimmed_df.drop('period_begin','period_end','median_sale_price','median_ppsf').distinct().sort(housing_trimmed_df.state.desc())
housing_transformed_5 = housing_transformed_4.join(housing_unique,(housing_unique.region==housing_transformed_4.zip) & (housing_unique.property_type_id==housing_transformed_4.type_id),'left')
housing_transformed_6 = housing_transformed_5.withColumn('median_sale_price_delta',housing_transformed_5['sum(median_sales_2020)'] - housing_transformed_5['sum(median_sales_2012)']).withColumn('median_ppsf_delta',housing_transformed_5['sum(median_ppsf_2020)'] - housing_transformed_5['sum(median_ppsf_2012)']).withColumn('zip_only',split('zip',' ').getItem(2))
housing_transformed_7 = housing_transformed_6.where(col('sum(median_sales_2020)')!= 0.0).where(col('sum(median_sales_2012)')!= 0.0).where(col('sum(median_ppsf_2020)')!= 0.0).where(col('sum(median_ppsf_2012)')!= 0.0)
housing_transformed_8 = housing_transformed_7.withColumn('Investment_Value_Sales',\
                                                         when(housing_transformed_7.median_sale_price_delta / housing_transformed_7['sum(median_sales_2012)'] < 0.05, 'Bad')
                                                         .when(housing_transformed_7.median_sale_price_delta / housing_transformed_7['sum(median_sales_2012)'] <= 0.10, 'Good')
                                                         .when(housing_transformed_7.median_sale_price_delta / housing_transformed_7['sum(median_sales_2012)'] <= 0.20, 'Great')
                                                         .when(housing_transformed_7.median_sale_price_delta / housing_transformed_7['sum(median_sales_2012)'] > 0.20, 'Excellent')).withColumn('Investment_Value_Ppsf',\
                                                         when(housing_transformed_7.median_ppsf_delta / housing_transformed_7['sum(median_ppsf_2012)'] < 0.05, 'Bad')
                                                         .when(housing_transformed_7.median_ppsf_delta / housing_transformed_7['sum(median_ppsf_2012)'] <= 0.10, 'Good')
                                                         .when(housing_transformed_7.median_ppsf_delta / housing_transformed_7['sum(median_ppsf_2012)'] <= 0.20, 'Great')
                                                         .when(housing_transformed_7.median_ppsf_delta / housing_transformed_7['sum(median_ppsf_2012)'] > 0.20, 'Excellent'))
housing_transformed_final= housing_transformed_8.select(housing_transformed_7.zip_only.alias('housing_data_zip'),'state','state_code','property_type',housing_transformed_7.property_type_id.cast("int"),'median_sale_price_delta','median_ppsf_delta','Investment_Value_Sales','Investment_Value_Ppsf').coalesce(50)
housing_transformed_final.show(5)

+----------------+-------------+----------+--------------------+----------------+-----------------------+------------------+----------------------+---------------------+
|housing_data_zip|        state|state_code|       property_type|property_type_id|median_sale_price_delta| median_ppsf_delta|Investment_Value_Sales|Investment_Value_Ppsf|
+----------------+-------------+----------+--------------------+----------------+-----------------------+------------------+----------------------+---------------------+
|           01432|Massachusetts|        MA|         Condo/Co-op|               3|               544012.5|301.46480560302734|             Excellent|                Great|
|           01541|Massachusetts|        MA|     All Residential|              -1|              2363000.0| 617.3615188598633|             Excellent|            Excellent|
|           01778|Massachusetts|        MA|Multi-Family (2-4...|               4|              1202500.0| 566.3466644287109|             Excellent|   

In [ ]:
## https://api.census.gov/data/2020/acs/acs5/variables.html lists all the possible variable keys for the API Request
## Below provides a DF with the ZIP Code and the Median household income.
## Need to initialize spark session first
##CODE
import requests
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
import pandas as pd
from pyspark.sql.functions import col

start_year = 2012
end_year = 2017
variables = "B19013_001E,B01002_001E,B01003_001E,B09001_001E,B11009_002E"
year = start_year
key  = '46d133e794f5998e012ea4e0d83d6fb6085136b0'
geography = 'zip%20code%20tabulation%20area'

call = 1

while call < 3:
  census_API_call = requests.get(f'https://api.census.gov/data/{year}/acs/acs5?get={variables}&for={geography}:*&key={key}')
  census_JSON = census_API_call.json()
  df = pd.DataFrame(census_JSON[1:], columns = census_JSON[0])
  #print(df)
  sparkDF = spark.createDataFrame(df)
  #sparkDF.show()
  column_title_median_income = f"Median_Inc_{year}"
  column_title_median_age = f"Median_Age_{year}"
  column_title_total_pop = f"Total_Pop_{year}"
  column_title_num_children = f"Total_Kids_{year}"
  column_title_married_households = f"Total_Married_{year}"

  if year == start_year:
    updated_df = sparkDF.select(col("zip code tabulation area").alias("Zip_Code"),col("B19013_001E").alias(column_title_median_income)\
                                ,col("B01002_001E").alias(column_title_median_age)
                                ,col("B01003_001E").alias(column_title_total_pop)
                                ,col("B09001_001E").alias(column_title_num_children)
                                ,col("B11009_002E").alias(column_title_married_households))
  else:
    updated_df = updated_df.join(sparkDF,sparkDF['zip code tabulation area']== updated_df.Zip_Code,'left').select(updated_df ["*"],col("B19013_001E").alias(column_title_median_income)\
                                ,col("B01002_001E").alias(column_title_median_age)
                                ,col("B01003_001E").alias(column_title_total_pop)
                                ,col("B09001_001E").alias(column_title_num_children)
                                ,col("B11009_002E").alias(column_title_married_households))
  
  #print(year)
  year = end_year
  call += 1

updated_df.show()

In [16]:
from pyspark.sql.functions import countDistinct, col
census_median_inc = updated_df.withColumnRenamed('Zip_Code', 'Zip_Code_Census')\
.withColumnRenamed('Median_Inc_2012', 'Median_Income_2012').withColumnRenamed('Median_Inc_2017', 'Median_Income_2017')
census_median_inc_2 =census_median_inc.where(col('Median_Income_2012') != -666666666)\
.where(col('Median_Age_2012') != -666666666)\
.where(col('Total_Pop_2012') != -666666666)\
.where(col('Total_Kids_2012') != -666666666)\
.where(col('Total_Married_2012') != -666666666)\
.where(col('Median_Income_2017') != -666666666)\
.where(col('Median_Age_2017') != -666666666)\
.where(col('Total_Pop_2017') != -666666666)\
.where(col('Total_Kids_2017') != -666666666)\
.where(col('Total_Married_2017') != -666666666)

census_data_raw = census_median_inc_2.dropna()
census_data_raw.show()

+---------------+------------------+---------------+--------------+---------------+------------------+------------------+---------------+--------------+---------------+------------------+
|Zip_Code_Census|Median_Income_2012|Median_Age_2012|Total_Pop_2012|Total_Kids_2012|Total_Married_2012|Median_Income_2017|Median_Age_2017|Total_Pop_2017|Total_Kids_2017|Total_Married_2017|
+---------------+------------------+---------------+--------------+---------------+------------------+------------------+---------------+--------------+---------------+------------------+
|          02053|            108181|           41.4|         12742|           3432|                99|            115432|           41.2|         13162|           3318|               120|
|          02090|            130824|           44.2|         14616|           4323|               143|            146042|           45.0|         15612|           4184|               159|
|          03442|             65048|           39.2|        

In [18]:
census_data_transorm_1 = census_data_raw.withColumn('Median_Income_Delta', (census_data_raw.Median_Income_2017 - census_data_raw.Median_Income_2012))\
.withColumn('Median_Age_Delta', (census_data_raw.Median_Age_2017 - census_data_raw.Median_Age_2012))\
.withColumn('Total_Pop_Delta', (census_data_raw.Total_Pop_2017 - census_data_raw.Total_Pop_2012))\
.withColumn('Total_Kids_Delta', (census_data_raw.Total_Kids_2017 - census_data_raw.Total_Kids_2012))\
.withColumn('Total_Married_Delta', (census_data_raw.Total_Married_2017 - census_data_raw.Total_Married_2012))

census_data_deltas = census_data_transorm_1.select(census_data_transorm_1.Zip_Code_Census,census_data_transorm_1.Median_Income_Delta.cast('float'),census_data_transorm_1.Median_Age_Delta.cast('float'),census_data_transorm_1.Total_Pop_Delta.cast('float'),census_data_transorm_1.Total_Kids_Delta.cast('float'),census_data_transorm_1.Total_Married_Delta.cast('float'))
census_data_deltas.show()
census_data_deltas.printSchema()

+---------------+-------------------+----------------+---------------+----------------+-------------------+
|Zip_Code_Census|Median_Income_Delta|Median_Age_Delta|Total_Pop_Delta|Total_Kids_Delta|Total_Married_Delta|
+---------------+-------------------+----------------+---------------+----------------+-------------------+
|          02053|             7251.0|            -0.2|          420.0|          -114.0|               21.0|
|          02090|            15218.0|             0.8|          996.0|          -139.0|               16.0|
|          03442|            -5167.0|            -0.4|           27.0|           -26.0|                2.0|
|          03904|            19091.0|            -4.5|         -175.0|           160.0|              -97.0|
|          04438|             2396.0|             1.6|         -363.0|          -138.0|              -13.0|
|          06382|             5884.0|             2.4|          347.0|          -403.0|              178.0|
|          06518|           

In [19]:
# Join two dataframes to reflect the sales price and median income based on the zipcode
df_merged = housing_transformed_final.join(census_data_deltas, census_data_deltas ['Zip_Code_Census'] ==  housing_transformed_final['housing_data_zip'],'left')\
                                                .select(housing_transformed_final['housing_data_zip'].alias('Zip_Code'), 
                                                housing_transformed_final['state'],
                                                housing_transformed_final['state_code'],
                                                housing_transformed_final['property_type'],
                                                housing_transformed_final['property_type_id'],
                                                housing_transformed_final['median_sale_price_delta'],
                                                housing_transformed_final['median_ppsf_delta'],
                                                housing_transformed_final['Investment_Value_Sales'],
                                                housing_transformed_final['Investment_Value_Ppsf'],
                                                census_data_deltas['Median_Income_Delta'],
                                                census_data_deltas['Median_Age_Delta'],
                                                census_data_deltas['Total_Pop_Delta'],
                                                census_data_deltas['Total_Kids_Delta'],
                                                census_data_deltas['Total_Married_Delta'])
df_merged = df_merged.dropna()
df_merged.coalesce(50)


DataFrame[Zip_Code: string, state: string, state_code: string, property_type: string, property_type_id: int, median_sale_price_delta: double, median_ppsf_delta: double, Investment_Value_Sales: string, Investment_Value_Ppsf: string, Median_Income_Delta: float, Median_Age_Delta: float, Total_Pop_Delta: float, Total_Kids_Delta: float, Total_Married_Delta: float]

In [74]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['property_type_id','Median_Income_Delta','Median_Age_Delta','Total_Pop_Delta','Total_Kids_Delta','Total_Married_Delta'], outputCol = 'features')
vectored_df = vectorAssembler.transform(df_merged)

In [25]:
(training_LR, test_LR) = vectored_df.randomSplit([0.8, 0.2], seed = 1234)

In [28]:
#Linear Regression for Median Sales Price
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='median_sale_price_delta', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model_sales_price = lr.fit(training_LR)
training_predictions=lr_model_sales_price.evaluate(training_LR)
test_prediction  = lr_model_sales_price.evaluate(test_LR)

print("Coefficients: " + str(lr_model_sales_price.coefficients))
print("Intercept: " + str(lr_model_sales_price.intercept))
print("Training R squared:", training_predictions.r2)
print("Testing MSE: ", test_prediction.meanSquaredError)


Coefficients: [-3099.258130930239,30.6263860211936,7294.6687792028815,187.13652912018827,-313.62245189701287,-566.0278153120107]
Intercept: 1347221.117336493
Training R squared: 0.04112565384203526
Testing MSE:  3400000552411.435


In [ ]:
#Linear Regression for Price Per Square Foot
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='median_ppsf_delta', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model_ppsf = lr.fit(training_LR)
training_predictions=lr_model_ppsf.evaluate(training_LR)
test_prediction  = lr_model_ppsf.evaluate(test_LR)

print("Coefficients: " + str(lr_model_sales_price.coefficients))
print("Intercept: " + str(lr_model_sales_price.intercept))
print("Training R squared:", training_predictions.r2)
print("Testing MSE: ", test_prediction.meanSquaredError)

In [37]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
indexer_sales = StringIndexer(inputCol = 'Investment_Value_Sales', outputCol = 'label')
df_features_indexed_sales = indexer_sales.fit(vectored_df).transform(vectored_df)

In [38]:
(training_ann_sales, test_ann_sales) = df_features_indexed_sales.randomSplit([0.8, 0.2], seed = 1234)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

mlp_sales = MultilayerPerceptronClassifier(layers = [6,5,5,4], seed = 1)

mlp_model_sales = mlp_sales.fit(training_ann_sales)

df_pred_sales = mlp_model_sales.transform(test_ann_sales)
df_pred_sales = df_pred_sales#.select('features','label','rawPrediction','probability','prediction')
df_pred_sales.show()

evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'accuracy')
mlpacc_sales = evaluator.evaluate(df_pred_sales)
print(mlpacc_sales)

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
indexer_ppsf = StringIndexer(inputCol = 'Investment_Value_Ppsf', outputCol = 'label')
df_features_indexed_ppsf = indexer_ppsf.fit(vectored_df).transform(vectored_df)

In [ ]:
(training_ann_ppsf, test_ann_ppsf) = df_features_indexed_ppsf.randomSplit([0.8, 0.2], seed = 1234)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

mlp = MultilayerPerceptronClassifier(layers = [6,5,5,4], seed = 1)

mlp_model = mlp.fit(training_ann_ppsf)

df_pred_ppsf = mlp_model.transform(test_ann_ppsf)
df_pred_ppsf = df_pred_ppsf.select('features','label','rawPrediction','probability','prediction')
df_pred_ppsf.show(truncate=False)

evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'accuracy')
mlpacc = evaluator.evaluate(df_pred_ppsf)
print(mlpacc)

In [76]:
## https://api.census.gov/data/2020/acs/acs5/variables.html lists all the possible variable keys for the API Request
## Below provides a DF with the ZIP Code and the Median household income.
## Need to initialize spark session first
##CODE
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import requests
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
import pandas as pd
from pyspark.sql.functions import col

start_year = 2015
end_year = 2018
variables = "B19013_001E,B01002_001E,B01003_001E,B09001_001E,B11009_002E"
year = start_year
key  = '46d133e794f5998e012ea4e0d83d6fb6085136b0'
geography = 'zip%20code%20tabulation%20area'

call = 1

while call < 3:
  census_API_call = requests.get(f'https://api.census.gov/data/{year}/acs/acs5?get={variables}&for={geography}:*&key={key}')
  census_JSON = census_API_call.json()
  df = pd.DataFrame(census_JSON[1:], columns = census_JSON[0])
  #print(df)
  schema = StructType([ StructField("B19013_001E", StringType(), True), StructField("B01002_001E", StringType(), True), StructField("B01003_001E", StringType(), True), StructField("B09001_001E", StringType(), True), StructField("B11009_002E", StringType(), True),StructField("State", StringType(), True),StructField("zip code tabulation area", StringType(), True)])
  sparkDF = spark.createDataFrame(df,schema = schema)
  #sparkDF.show()
  column_title_median_income = f"Median_Inc_{year}"
  column_title_median_age = f"Median_Age_{year}"
  column_title_total_pop = f"Total_Pop_{year}"
  column_title_num_children = f"Total_Kids_{year}"
  column_title_married_households = f"Total_Married_{year}"

  if year == start_year:
    real_df = sparkDF.select(col("zip code tabulation area").alias("Zip_Code"),col("B19013_001E").alias(column_title_median_income)\
                                ,col("B01002_001E").alias(column_title_median_age)
                                ,col("B01003_001E").alias(column_title_total_pop)
                                ,col("B09001_001E").alias(column_title_num_children)
                                ,col("B11009_002E").alias(column_title_married_households))
  
  else:
    real_df = real_df.join(sparkDF,sparkDF['zip code tabulation area']== real_df.Zip_Code,'left').select(real_df ["*"],col("B19013_001E").alias(column_title_median_income)\
                                ,col("B01002_001E").alias(column_title_median_age)
                                ,col("B01003_001E").alias(column_title_total_pop)
                                ,col("B09001_001E").alias(column_title_num_children)
                                ,col("B11009_002E").alias(column_title_married_households))
  
  #print(year)
  year = end_year
  call += 1

real_df.show()


+--------+---------------+---------------+--------------+---------------+------------------+---------------+---------------+--------------+---------------+------------------+
|Zip_Code|Median_Inc_2015|Median_Age_2015|Total_Pop_2015|Total_Kids_2015|Total_Married_2015|Median_Inc_2018|Median_Age_2018|Total_Pop_2018|Total_Kids_2018|Total_Married_2018|
+--------+---------------+---------------+--------------+---------------+------------------+---------------+---------------+--------------+---------------+------------------+
|   02053|         110241|           41.7|         13069|           3275|                87|         119450|           42.6|         13244|           3279|               147|
|   02090|         135884|           45.8|         15094|           4115|               151|         155887|           44.9|         15880|           4134|               172|
|   03442|          63958|           35.8|          1424|            447|                13|          61818|           38.2| 

In [77]:
from pyspark.sql.functions import countDistinct, col
census_real = real_df.withColumnRenamed('Zip_Code', 'Zip_Code_Census')\
.withColumnRenamed('Median_Inc_2015', 'Median_Income_2015').withColumnRenamed('Median_Inc_2018', 'Median_Income_2018')
census_real_3 =census_real.where(col('Median_Income_2015') != -666666666)\
.where(col('Median_Age_2015') != -666666666)\
.where(col('Total_Pop_2015') != -666666666)\
.where(col('Total_Kids_2015') != -666666666)\
.where(col('Total_Married_2015') != -666666666)\
.where(col('Median_Income_2018') != -666666666)\
.where(col('Median_Age_2018') != -666666666)\
.where(col('Total_Pop_2018') != -666666666)\
.where(col('Total_Kids_2018') != -666666666)\
.where(col('Total_Married_2018') != -666666666)

census_data_raw_real = census_real_3.dropna()
census_data_raw_real.show()
#census_real_3.show()

+---------------+------------------+---------------+--------------+---------------+------------------+------------------+---------------+--------------+---------------+------------------+
|Zip_Code_Census|Median_Income_2015|Median_Age_2015|Total_Pop_2015|Total_Kids_2015|Total_Married_2015|Median_Income_2018|Median_Age_2018|Total_Pop_2018|Total_Kids_2018|Total_Married_2018|
+---------------+------------------+---------------+--------------+---------------+------------------+------------------+---------------+--------------+---------------+------------------+
|          02053|            110241|           41.7|         13069|           3275|                87|            119450|           42.6|         13244|           3279|               147|
|          02090|            135884|           45.8|         15094|           4115|               151|            155887|           44.9|         15880|           4134|               172|
|          03442|             63958|           35.8|        

In [78]:
census_data_real_transorm_1 = census_data_raw_real.withColumn('Median_Income_Delta', (census_data_raw_real.Median_Income_2018 - census_data_raw_real.Median_Income_2015))\
.withColumn('Median_Age_Delta', (census_data_raw_real.Median_Age_2018 - census_data_raw_real.Median_Age_2015))\
.withColumn('Total_Pop_Delta', (census_data_raw_real.Total_Pop_2018 - census_data_raw_real.Total_Pop_2015))\
.withColumn('Total_Kids_Delta', (census_data_raw_real.Total_Kids_2018 - census_data_raw_real.Total_Kids_2015))\
.withColumn('Total_Married_Delta', (census_data_raw_real.Total_Married_2018 - census_data_raw_real.Total_Married_2015))

census_data_deltas_real = census_data_real_transorm_1.select(census_data_real_transorm_1.Zip_Code_Census,census_data_real_transorm_1.Median_Income_Delta.cast('float'),census_data_real_transorm_1.Median_Age_Delta.cast('float'),census_data_real_transorm_1.Total_Pop_Delta.cast('float'),census_data_real_transorm_1.Total_Kids_Delta.cast('float'),census_data_real_transorm_1.Total_Married_Delta.cast('float'))
census_data_deltas_real.show()
census_data_deltas_real.printSchema()

+---------------+-------------------+----------------+---------------+----------------+-------------------+
|Zip_Code_Census|Median_Income_Delta|Median_Age_Delta|Total_Pop_Delta|Total_Kids_Delta|Total_Married_Delta|
+---------------+-------------------+----------------+---------------+----------------+-------------------+
|          02053|             9209.0|             0.9|          175.0|             4.0|               60.0|
|          02090|            20003.0|            -0.9|          786.0|            19.0|               21.0|
|          03442|            -2140.0|             2.4|          -17.0|          -145.0|               55.0|
|          03904|             7768.0|            -3.5|         -103.0|            58.0|               12.0|
|          04438|             5781.0|             1.5|          -96.0|           -76.0|               11.0|
|          06382|             9645.0|             0.2|          244.0|            25.0|               11.0|
|          06518|           

In [81]:
property_type_id = df_merged.select("property_type_id",'property_type').distinct()
property_type_id.show()

+----------------+--------------------+
|property_type_id|       property_type|
+----------------+--------------------+
|              13|           Townhouse|
|              -1|     All Residential|
|               6|Single Family Res...|
|               4|Multi-Family (2-4...|
|               3|         Condo/Co-op|
+----------------+--------------------+



In [86]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col,lit
prop_type_id = '6'
census_data_deltas_real = census_data_deltas_real.withColumn('prop_type_id', lit(prop_type_id))
vectorAssembler = VectorAssembler(inputCols = ['prop_type_id','Median_Income_Delta','Median_Age_Delta','Total_Pop_Delta','Total_Kids_Delta','Total_Married_Delta'], outputCol = 'features')
vectored_df = vectorAssembler.transform(census_data_deltas_real)

In [87]:
df_pred_sales = mlp_model_sales.transform(vectored_df)
df_pred_sales = df_pred_sales#.select('features','label','rawPrediction','probability','prediction')
df_pred_sales.show()


+---------------+-------------------+----------------+---------------+----------------+-------------------+------------+--------------------+--------------------+--------------------+----------+
|Zip_Code_Census|Median_Income_Delta|Median_Age_Delta|Total_Pop_Delta|Total_Kids_Delta|Total_Married_Delta|prop_type_id|            features|       rawPrediction|         probability|prediction|
+---------------+-------------------+----------------+---------------+----------------+-------------------+------------+--------------------+--------------------+--------------------+----------+
|          02053|             9209.0|             0.9|          175.0|             4.0|               60.0|           6|[6.0,9209.0,0.899...|[2.64895421257875...|[0.91797709476904...|       0.0|
|          02090|            20003.0|            -0.9|          786.0|            19.0|               21.0|           6|[6.0,20003.0,-0.8...|[2.64895421257875...|[0.91797709476904...|       0.0|
|          03442|        

In [90]:
df_pred_sales.select('prediction').distinct().show()

+----------+
|prediction|
+----------+
|       0.0|
+----------+

